In [1]:
## freebayes is good variant caller but it takes BAM file
####to do this conversion we can use samtools
! ~/miniconda3/bin/freebayes --help
! ~/miniconda3/bin/samtools view -S -b /mnt/d/variant_call_example/Coriell-NA12877.sam > /mnt/d/variant_call_example/Coriell-NA12877.bam

usage: /home/joewu/miniconda3/bin/freebayes [OPTION] ... [BAM FILE] ... 

Bayesian haplotype-based polymorphism discovery.

citation: Erik Garrison, Gabor Marth
          "Haplotype-based variant detection from short-read sequencing"
          arXiv:1207.3907 (http://arxiv.org/abs/1207.3907)

overview:

    To call variants from aligned short-read sequencing data, supply BAM files and
    a reference.  FreeBayes will provide VCF output on standard out describing SNPs,
    indels, and complex variants in samples in the input alignments.

    By default, FreeBayes will consider variants supported by at least 2
    observations in a single sample (-C) and also by at least 20% of the reads from
    a single sample (-F).  These settings are suitable to low to high depth
    sequencing in haploid and diploid samples, but users working with polyploid or
    pooled samples may wish to adjust them depending on the characteristics of
    their sequencing data.

    FreeBayes is capable of callin

In [2]:
! ~/miniconda3/bin/samtools sort /mnt/d/variant_call_example/Coriell-NA12877.bam -o /mnt/d/variant_call_example/Coriell-NA12877-sorted.bam
! ~/miniconda3/bin/samtools index /mnt/d/variant_call_example/Coriell-NA12877-sorted.bam

In [5]:
! ~/miniconda3/bin/freebayes -f /mnt/d/genomes/Homo_sapiens/UCSC/hg38/Sequence/Bowtie2Index/genome.fa /mnt/d/variant_call_example/Coriell-NA12877-sorted.bam > /mnt/d/variant_call_example/Coriell-NA12877.vcf

In [16]:
## the output of freebayes is a variant call file (*.vcf) - Coriell-NA12877.vcf
####for more on vcf see https://gatk.broadinstitute.org/hc/en-us/articles/360035531692-VCF-Variant-Call-Format

! ls /mnt/d/variant_call_example

Coriell-NA12877.bam			   Coriell-NA12877_R2_fastqc.html
Coriell-NA12877_R1.fastq		   Coriell-NA12877_R2_fastqc.zip
Coriell-NA12877_R1_fastqc		   Coriell-NA12877.sam
Coriell-NA12877_R1_fastqc.html		   Coriell-NA12877-sorted.bam
Coriell-NA12877_R1_fastqc_image_files.txt  Coriell-NA12877-sorted.bam.bai
Coriell-NA12877_R1_fastqc.zip		   Coriell-NA12877.vcf
Coriell-NA12877_R2.fastq		   quality_check_demo.ipynb
Coriell-NA12877_R2_fastqc


In [22]:
## briefly, the vcf file will tell us genomic position along with gentoype
####differences between sample and reference and quality of the variant call (QUAL),
####where QUAL is defined as -10log_base10(probability that a variant does not exist)
####the higher the QUAL, the more confident we are about the variant call
####as a downstream process, we can choose to filter out the low QUAL variant
####calls in a vcf
## we can also run the applications such as SNPeff to get annotations on
####the variant calls 

! sed -n 56,71p /mnt/d/variant_call_example/Coriell-NA12877.vcf | cut -f1-6

#CHROM	POS	ID	REF	ALT	QUAL
chr1	162807214	.	A	C	2.69375
chr1	162807221	.	G	C	2.69112
chr1	162807242	.	T	C	2.69961
chr3	130098396	.	C	G	2.70097
chr4	127812999	.	T	G	43.8752
chr4	127813032	.	A	G	29.7786
chr9	138139684	.	A	G	0.848001
chr9	138139725	.	T	C	2.61557
chr9	138139738	.	G	A	2.6488
chr9	138139772	.	TGGT	GGGC	0.384476
chr9	138139943	.	C	G	0.634019
chr9	138139978	.	A	G	3.13129
chr10	6804696	.	G	T	1.97689
chr10	28899339	.	T	A	0.401488
chr10	28899362	.	G	T	0.401488


In [34]:
## here we can us vcftools to extract those variant calls with a
####QUAL of 40 or greater and save as Coriell-NA12877-QUAL40.recode.vcf

! vcftools --vcf /mnt/d/variant_call_example/Coriell-NA12877.vcf --minQ 40 --recode --recode-INFO-all --out /mnt/d/variant_call_example/Coriell-NA12877-QUAL40


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf /mnt/d/variant_call_example/Coriell-NA12877.vcf
	--recode-INFO-all
	--minQ 40
	--out /mnt/d/variant_call_example/Coriell-NA12877-QUAL40
	--recode

After filtering, kept 1 out of 1 Individuals
Outputting VCF file...
After filtering, kept 1180 out of a possible 1789 Sites
Run Time = 2.00 seconds


In [35]:
! ls /mnt/d/variant_call_example/

Coriell-NA12877.bam			   Coriell-NA12877_R2.fastq
Coriell-NA12877-QUAL40.log		   Coriell-NA12877_R2_fastqc
Coriell-NA12877-QUAL40.recode.vcf	   Coriell-NA12877_R2_fastqc.html
Coriell-NA12877-QUAL40.vcf.log		   Coriell-NA12877_R2_fastqc.zip
Coriell-NA12877_R1.fastq		   Coriell-NA12877.sam
Coriell-NA12877_R1_fastqc		   Coriell-NA12877-sorted.bam
Coriell-NA12877_R1_fastqc.html		   Coriell-NA12877-sorted.bam.bai
Coriell-NA12877_R1_fastqc_image_files.txt  Coriell-NA12877.vcf
Coriell-NA12877_R1_fastqc.zip		   quality_check_demo.ipynb


In [36]:
! sed -n 56,71p /mnt/d/variant_call_example/Coriell-NA12877-QUAL40.recode.vcf | cut -f1-6

#CHROM	POS	ID	REF	ALT	QUAL
chr4	127812999	.	T	G	43.8752
chr16	29609	.	C	G	2196.38
chr16	34170	.	C	T	6532.29
chr16	34870	.	C	G	842.8
chr16	35667	.	C	A	2671.02
chr16	58586	.	T	C	284.73
chr16	62515	.	T	C	57.3966
chr16	78180	.	T	C	231.236
chr16	80894	.	G	C	56.6956
chr16	81553	.	CTG	CG	52.9304
chr16	81566	.	C	A	56.6956
chr16	83614	.	T	A	40.2077
chr16	86143	.	C	T	51.376
chr16	92283	.	G	T	54.3723
chr16	98354	.	G	A	57.3966
